In [1]:
import pyarrow

In [3]:
# Import the libraries we will need
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

import findspark
findspark.init()

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
import pyspark.pandas as ps

# create the Spark Session
spark = SparkSession.builder.appName("Q1").getOrCreate()

# # create the Spark Context
# sc = spark.sparkContext

c:\Users\Wenhao\miniconda3\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# friend = spark.read.csv("hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt", sep='\t')
# friend = friend.select(col("_c0").alias("User"), col("_c1").alias("Friends"))
# friend = friend.withColumn('Friends', split('Friends', ','))
# friend.show(5)

In [39]:
friend = spark.read.csv("hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt", sep='\t')
friend = friend.toDF("User", "Friends")
friend = friend.withColumn("User", friend["User"].cast("int"))
friend = friend.withColumn("Friends", split(friend["Friends"], ",").cast("array<int>"))

no_friend = friend.filter(size(friend["Friends"]) == -1).withColumnRenamed("Friends", "Recommendations")

friend = friend.filter(size(friend["Friends"]) != -1)

In [12]:
# friend = spark.read.csv("hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt", sep='\t')
# friend = friend.pandas_api()
# friend = friend.rename({'_c0': 'User', '_c1': 'Friends'}, axis = 1)
# friend['User'] = friend['User'].astype(int)
# friend['Friends'] = friend['Friends'].apply(lambda x: [] if (x is None) else x.split(',')) # split the index of friends into list

# no_friend = friend[friend['Friends'].apply(len) == 0].rename({'Friends': 'Recommendations'}, axis = 1) # filter users who have no friend

# friend = friend[friend['Friends'].apply(len) > 0] # filter users who have at least one friend
# friend = friend.iloc[:100] # FIX ME: SUBSAMPLING
# friend.head(5)

In [40]:
all_user_ids = friend.select("User").rdd.flatMap(lambda x:x).collect()
all_user_ids = spark.createDataFrame([(all_user_ids,)], ["All Users"])
friend = friend.crossJoin(all_user_ids)

In [41]:
# all_user_id = friend.select("User")
unfriend = friend.withColumn("Unfriends", array_except(friend["All Users"], friend["Friends"])).select("User", "Unfriends")
unfriend = unfriend.withColumn("Unfriend", explode(unfriend["Unfriends"]))
unfriend = unfriend.withColumn("Unfriend", unfriend["Unfriend"].cast("int"))
unfriend = unfriend.filter(unfriend["User"] != unfriend["Unfriend"])

friend = friend.select(["User", "Friends"])

In [50]:
mutual_friend = unfriend.join(friend, on="User", how="left").withColumnRenamed("Friends", "User's Friends")
mutual_friend = mutual_friend.join(friend.withColumnRenamed("User", "Unfriend"), on="Unfriend", how="left").withColumnRenamed("Friends", "Unfriend's Friends")
mutual_friend = mutual_friend.withColumn("Mutual Friends", array_intersect(mutual_friend["User's Friends"], mutual_friend["Unfriend's Friends"]))
mutual_friend = mutual_friend.withColumn("Num of Mutual Friends", size(mutual_friend["Mutual Friends"]))
mutual_friend = mutual_friend.select(["User", "Unfriend", "Num of Mutual Friends"])


In [51]:
window_spec = Window.partitionBy("User").orderBy(desc("Num of Mutual Friends"), asc("Unfriend"))

In [54]:
mutual_friend = mutual_friend.withColumn("rank", row_number().over(window_spec))
mutual_friend = mutual_friend.filter(mutual_friend["rank"] <= 10)

In [55]:
mutual_friend.filter(mutual_friend["User"] == 11).show(20)

+----+--------+---------------------+----+
|User|Unfriend|Num of Mutual Friends|rank|
+----+--------+---------------------+----+
|  11|   27552|                    4|   1|
|  11|    7785|                    3|   2|
|  11|   27573|                    3|   3|
|  11|   27574|                    3|   4|
|  11|   27589|                    3|   5|
|  11|   27590|                    3|   6|
|  11|   27600|                    3|   7|
|  11|   27617|                    3|   8|
|  11|   27620|                    3|   9|
|  11|   27667|                    3|  10|
+----+--------+---------------------+----+

